In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Carregar CSVs como DataFrames pandas
df_dict = pd.read_csv(r'C:\Users\Aluno\Documents\GitHub\final-project\data\P18_dicionario_dados.csv', encoding='utf-8', sep=',')
df_cancel = pd.read_csv(r'C:\Users\Aluno\Documents\GitHub\final-project\data\P18_cancelamentos.csv', encoding='utf-8', sep=',')
df_contratos = pd.read_csv(r'C:\Users\Aluno\Documents\GitHub\final-project\data\P18_contratos.csv', encoding='utf-8', sep=',')
df_clientes = pd.read_csv(r'C:\Users\Aluno\Documents\GitHub\final-project\data\P18_clientes.csv', encoding='utf-8', sep=',')

# Criar banco SQLite em memória (ou em disco se preferir salvar como arquivo)
conn = sqlite3.connect(":memory:")  # ou 'meubanco.db' se quiser salvar em disco

# Registrar os DataFrames como tabelas SQL dentro do SQLite
df_cancel.to_sql("cancelamentos", conn, index=False, if_exists="replace")
df_contratos.to_sql("contratos", conn, index=False, if_exists="replace")
df_clientes.to_sql("clientes", conn, index=False, if_exists="replace")

5000

In [3]:
import sys
print(sys.executable)

c:\ProgramData\anaconda3\python.exe


1. Quantos planos de seguro temos?

In [4]:
df_contratos['tipo_seguro'].nunique()
df_contratos['tipo_seguro'].value_counts()

tipo_seguro
Auto           2429
Empresarial    2423
Vida           2406
Saúde          2390
Residencial    2352
Name: count, dtype: int64

2. Quais planos possuem mais cancelamento?

In [5]:
# Unir Contratos com Cancelamentos
df_cancelados = df_contratos.merge(df_cancel, on='id_contrato', how='inner')

# Contar Cancelamentos por Tipo de Seguro
df_cancelados['tipo_seguro'].value_counts()

tipo_seguro
Saúde          528
Empresarial    522
Residencial    512
Vida           489
Auto           469
Name: count, dtype: int64

3. Qual o tempo de contrato médio dos clientes?

In [6]:
# Converter Datas
df_contratos['data_inicio'] = pd.to_datetime(df_contratos['data_inicio'])
df_contratos['data_fim'] = pd.to_datetime(df_contratos['data_fim'])

# Criar Coluna com Duração em Dias
df_contratos['duracao_contrato'] = (df_contratos['data_fim'] - df_contratos['data_inicio']).dt.days

# Média Geral
df_contratos['duracao_contrato'].mean()

424.2079166666667

4. Qual o tempo médio dos contratos cancelados?

In [7]:
df_cancelados['data_inicio'] = pd.to_datetime(df_cancelados['data_inicio'])
df_cancelados['data_fim'] = pd.to_datetime(df_cancelados['data_fim'])

df_cancelados['duracao_contrato'] = (df_cancelados['data_fim'] - df_cancelados['data_inicio']).dt.days
df_cancelados['duracao_contrato'].mean()

415.67857142857144

5. Houve troca de plano antes do cancelamento?

In [8]:
# Ver Clientes com Mais de um Tipo de Seguro
df_contratos.sort_values(by=['id_cliente', 'data_inicio'], inplace=True)
df_troca = df_contratos.groupby('id_cliente')['tipo_seguro'].nunique()

# Clientes que Já Tiveram Mais de um Tipo
trocaram = df_troca[df_troca > 1].index

# Quantos Desses Também Cancelaram
cancelaram_e_trocaram = df_cancelados[df_cancelados['id_cliente'].isin(trocaram)]
print("Cancelaram após trocar de plano:", len(cancelaram_e_trocaram))

Cancelaram após trocar de plano: 2184


6. Clientes que renovaram contrato ao menos uma vez cancelam menos?

In [9]:
# Clientes que Renovaram
renovaram = df_contratos[df_contratos['renovado_automaticamente'] == True]
nao_renovaram = df_contratos[df_contratos['renovado_automaticamente'] == False]

# Com Cancelamento
renovaram_cancelaram = renovaram[renovaram['id_contrato'].isin(df_cancel['id_contrato'])]
nao_renovaram_cancelaram = nao_renovaram[nao_renovaram['id_contrato'].isin(df_cancel['id_contrato'])]

# Taxas
print("Taxa de cancelamento com renovação:", len(renovaram_cancelaram) / len(renovaram))
print("Taxa sem renovação:", len(nao_renovaram_cancelaram) / len(nao_renovaram))

Taxa de cancelamento com renovação: 0.20939464108501488
Taxa sem renovação: 0.2106147127981189


7. Canal de venda influencia na fidelidade?

In [10]:
df_contratos['cancelado'] = df_contratos['id_contrato'].isin(df_cancel['id_contrato'])

df_contratos.groupby('canal_venda')['cancelado'].mean().sort_values(ascending=False)

canal_venda
Agência Física    0.211807
App               0.211277
Corretor          0.209721
Site              0.207192
Name: cancelado, dtype: float64

8. Avaliação da experiência no cancelamento x motivo de cancelamento

In [30]:
query = """
SELECT 
  motivo_cancelamento,
  ROUND(AVG(
    CASE 
      WHEN avaliacao_experiencia_cancelamento = 'Ruim' THEN 1
      WHEN avaliacao_experiencia_cancelamento = 'Neutra' THEN 2
      WHEN avaliacao_experiencia_cancelamento = 'Boa' THEN 3
      ELSE NULL
    END
  ), 2) AS media_avaliacao
FROM cancelamentos
GROUP BY motivo_cancelamento
ORDER BY media_avaliacao ASC
"""
df_avaliacao_motivo = pd.read_sql_query(query, conn)
print(df_avaliacao_motivo)

      motivo_cancelamento  media_avaliacao
0              Preço alto             1.99
1   Problemas no sinistro             2.01
2  Cobertura insuficiente             2.02
3        Atendimento ruim             2.05
4   Mudança de seguradora             2.05


9.  Clientes com baixa satisfação e baixa renda

In [44]:
query = """
SELECT 
  c.id_cliente,
  c.renda_mensal,
  co.satisfacao_ultima_avaliacao,
  ca.avaliacao_experiencia_cancelamento,
  ca.data_cancelamento
FROM cancelamentos ca
JOIN contratos co ON ca.id_contrato = co.id_contrato
JOIN clientes c ON co.id_cliente = c.id_cliente
WHERE 
  (
    CASE 
      WHEN co.satisfacao_ultima_avaliacao = 'Baixa' THEN 1
      WHEN co.satisfacao_ultima_avaliacao = 'Média' THEN 2
      WHEN co.satisfacao_ultima_avaliacao = 'Alta' THEN 3
      ELSE NULL
    END
  ) <= 2
  AND c.renda_mensal < 10000
"""
df_baixa_renda_satisfacao = pd.read_sql_query(query, conn)
print(df_baixa_renda_satisfacao)

    id_cliente  renda_mensal satisfacao_ultima_avaliacao  \
0       C01330       5506.98                       Baixa   
1       C00994       8500.11                       Baixa   
2       C04962       3191.25                       Baixa   
3       C02323       8192.83                       Baixa   
4       C03199       4977.14                       Baixa   
..         ...           ...                         ...   
768     C04700       4903.29                       Média   
769     C03009       6720.83                       Baixa   
770     C01688       3605.98                       Média   
771     C04662       8998.38                       Baixa   
772     C00312       3167.47                       Baixa   

    avaliacao_experiencia_cancelamento data_cancelamento  
0                                 Ruim        2024-11-04  
1                                 Ruim        2025-08-31  
2                               Neutra        2023-11-12  
3                               Neutra     

10. Cancelamento por tipo de plano e número de dependentes (Vida e Saúde)

In [45]:
query = """
SELECT 
  co.tipo_seguro,
  c.qtd_dependentes,
  COUNT(*) AS total_cancelamentos
FROM cancelamentos ca
JOIN contratos co ON ca.id_contrato = co.id_contrato
JOIN clientes c ON co.id_cliente = c.id_cliente
WHERE co.tipo_seguro IN ('Vida', 'Saúde')
GROUP BY co.tipo_seguro, c.qtd_dependentes
ORDER BY co.tipo_seguro, c.qtd_dependentes
"""
df_cancel_tipo_dependentes = pd.read_sql_query(query, conn)
print(df_cancel_tipo_dependentes)

   tipo_seguro  qtd_dependentes  total_cancelamentos
0        Saúde                0                  122
1        Saúde                1                  178
2        Saúde                2                  141
3        Saúde                3                   57
4        Saúde                4                   22
5        Saúde                5                    6
6        Saúde                6                    1
7        Saúde                7                    1
8         Vida                0                  122
9         Vida                1                  158
10        Vida                2                  134
11        Vida                3                   44
12        Vida                4                   25
13        Vida                5                    6


11. Tempo de cliente na base (em anos) x cancelamento

In [47]:
query = """
WITH base_tempo AS (
  SELECT 
    c.id_cliente,
    c.data_cadastro,
    ca.data_cancelamento,
    CASE 
      WHEN ca.data_cancelamento IS NOT NULL THEN julianday(ca.data_cancelamento) - julianday(c.data_cadastro)
      ELSE julianday('now') - julianday(c.data_cadastro)
    END AS dias_na_base,
    CASE 
      WHEN ca.id_contrato IS NOT NULL THEN 1
      ELSE 0
    END AS cancelou
  FROM clientes c
  LEFT JOIN contratos co ON c.id_cliente = co.id_cliente
  LEFT JOIN cancelamentos ca ON co.id_contrato = ca.id_contrato
)
SELECT
  cancelou,
  ROUND(AVG(dias_na_base)/365, 2) AS tempo_medio_anos,
  COUNT(*) AS qtd_clientes
FROM base_tempo
GROUP BY cancelou
ORDER BY cancelou DESC
"""
df_tempo_cancelamento = pd.read_sql_query(query, conn)
print(df_tempo_cancelamento)

   cancelou  tempo_medio_anos  qtd_clientes
0         1              1.73          2520
1         0              2.55          9931


12. Planos com risco (baixa satisfação + sem renovação automática)

In [48]:
query = """
SELECT 
  co.tipo_seguro,
  COUNT(*) AS qtd_planos_risco
FROM contratos co
WHERE co.satisfacao_ultima_avaliacao = 'Baixa'
  AND (co.renovado_automaticamente = 'Não' OR co.renovado_automaticamente = 0 OR co.renovado_automaticamente IS NULL)
GROUP BY co.tipo_seguro
ORDER BY qtd_planos_risco DESC
"""
df_planos_risco = pd.read_sql_query(query, conn)
print(df_planos_risco)

   tipo_seguro  qtd_planos_risco
0  Empresarial               426
1         Auto               412
2         Vida               405
3  Residencial               401
4        Saúde               394


13. Clientes que cancelaram pelo App: idade média e avaliação média

In [49]:
query = """
SELECT 
  ROUND(AVG((strftime('%Y', 'now') - strftime('%Y', c.data_nascimento))), 2) AS idade_media,
  ROUND(AVG(
    CASE 
      WHEN ca.avaliacao_experiencia_cancelamento = 'Boa' THEN 3
      WHEN ca.avaliacao_experiencia_cancelamento = 'Neutra' THEN 2
      WHEN ca.avaliacao_experiencia_cancelamento = 'Ruim' THEN 1
      ELSE NULL
    END
  ), 2) AS avaliacao_media
FROM cancelamentos ca
JOIN contratos co ON ca.id_contrato = co.id_contrato
JOIN clientes c ON co.id_cliente = c.id_cliente
WHERE ca.canal_cancelamento = 'App'
"""
df_cancel_app = pd.read_sql_query(query, conn)
print(df_cancel_app)

   idade_media  avaliacao_media
0        49.61             2.01


14.  Ranking proporcional de cancelamento por profissão

In [64]:
query = """
WITH total_clientes AS (
  SELECT profissao, COUNT(*) AS total
  FROM clientes
  GROUP BY profissao
),
cancelamentos_profissao AS (
  SELECT c.profissao, COUNT(DISTINCT co.id_cliente) AS total_cancelamentos
  FROM cancelamentos ca
  JOIN contratos co ON ca.id_contrato = co.id_contrato
  JOIN clientes c ON co.id_cliente = c.id_cliente
  GROUP BY c.profissao
)
SELECT
  tc.profissao,
  tc.total,
  COALESCE(cp.total_cancelamentos, 0) AS total_cancelamentos,
  ROUND(COALESCE(cp.total_cancelamentos, 0) * 1.0 / tc.total, 4) AS taxa_cancelamento
FROM total_clientes tc
LEFT JOIN cancelamentos_profissao cp ON tc.profissao = cp.profissao
ORDER BY taxa_cancelamento DESC, total_cancelamentos DESC
"""
df_ranking_profissao = pd.read_sql_query(query, conn)
print(df_ranking_profissao)

                               profissao  total  total_cancelamentos  \
0                      Lutador de karatê      6                    6   
1                                   Ator      5                    5   
2    Broker/Corretor da bolsa de valores      4                    4   
3                         Cerimonialista      4                    4   
4                              Figurante      4                    4   
..                                   ...    ...                  ...   
743       Tecnólogo em navegação fluvial      1                    0   
744            Tecnólogo em silvicultura      1                    0   
745             Vigilante noturno/diurno      5                    0   
746                          Webdesigner      5                    0   
747                   Árbitro e mediador      2                    0   

     taxa_cancelamento  
0                  1.0  
1                  1.0  
2                  1.0  
3                  1.0  
4         

15. Troca de plano durante o cancelamento

In [62]:
query = """
WITH tempo_cancelamento AS (
  SELECT 
    co.id_contrato,
    co.canal_venda,
    julianday(ca.data_cancelamento) - julianday(co.data_inicio) AS dias_ate_cancelar
  FROM cancelamentos ca
  JOIN contratos co ON ca.id_contrato = co.id_contrato
  WHERE ca.data_cancelamento IS NOT NULL
)
SELECT 
  canal_venda,
  ROUND(AVG(dias_ate_cancelar), 2) AS tempo_medio_dias,
  COUNT(*) AS total_cancelamentos
FROM tempo_cancelamento
GROUP BY canal_venda
ORDER BY tempo_medio_dias ASC;
"""

df_tempo_cancelamento_canal = pd.read_sql_query(query, conn)
print(df_tempo_cancelamento_canal.to_string(index=False))

   canal_venda  tempo_medio_dias  total_cancelamentos
           App            264.64                  622
          Site            265.79                  628
Agência Física            276.02                  653
      Corretor            281.26                  617


16. Tempo médio até cancelamento por canal de venda

In [65]:
query = """
WITH tempo_cancelamento AS (
  SELECT 
    co.id_contrato,
    co.canal_venda,
    julianday(ca.data_cancelamento) - julianday(co.data_inicio) AS dias_ate_cancelar
  FROM cancelamentos ca
  JOIN contratos co ON ca.id_contrato = co.id_contrato
  WHERE ca.data_cancelamento IS NOT NULL
)
SELECT 
  canal_venda,
  ROUND(AVG(dias_ate_cancelar), 2) AS tempo_medio_dias,
  COUNT(*) AS total_cancelamentos
FROM tempo_cancelamento
GROUP BY canal_venda
ORDER BY tempo_medio_dias ASC;
"""

df_tempo_cancelamento_canal = pd.read_sql_query(query, conn)
print(df_tempo_cancelamento_canal.to_string(index=False))


   canal_venda  tempo_medio_dias  total_cancelamentos
           App            264.64                  622
          Site            265.79                  628
Agência Física            276.02                  653
      Corretor            281.26                  617
